<a href="https://colab.research.google.com/github/thatsciencegal/flood_pheno/blob/main/SASA_PEAN_cc_rem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Initialize Google Earth Engine

In [2]:
import ee

## Trigger the authentication flow
ee.Authenticate()

## Initialize the library
ee.Initialize()

## Test the API by printing elevation of Mt. Everest
dem = ee.Image('USGS/SRTMGL1_003')
xy = ee.Geometry.Point([86.9250, 27.9881])
elev = dem.sample(xy, 30).first().get('elevation').getInfo()
print('Mount Everest elevation (m):', elev)

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=ttdx4Pa9Vzs8AIwtT9Ae_IOD5kN0di470FtmgkccKPA&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AY0e-g7ss2dh1cjCrJP27Bdu963Pd9dG9urXgaFBNzxZPV8icM73K103di8

Successfully saved authorization token.
Mount Everest elevation (m): 8729


Add mapping abilities

In [3]:
import folium

# Define the URL format used for Earth Engine generated map tiles.
EE_TILES = 'https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}'

#@title Mapdisplay: Display GEE objects using folium.
def Mapdisplay(center, dicc, Tiles="OpensTreetMap",zoom_start=10):
    '''
    :param center: Center of the map (Latitude and Longitude).
    :param dicc: Earth Engine Geometries or Tiles dictionary
    :param Tiles: Mapbox Bright,Mapbox Control Room,Stamen Terrain,Stamen Toner,stamenwatercolor,cartodbpositron.
    :zoom_start: Initial zoom level for the map.
    :return: A folium.Map object.
    '''
    mapViz = folium.Map(location=center,tiles=Tiles, zoom_start=zoom_start)
    for k,v in dicc.items():
      if ee.image.Image in [type(x) for x in v.values()]:
        folium.TileLayer(
            tiles = v["tile_fetcher"].url_format,
            attr  = 'Google Earth Engine',
            overlay =True,
            name  = k
          ).add_to(mapViz)
      else:
        folium.GeoJson(
        data = v,
        name = k
          ).add_to(mapViz)
    mapViz.add_child(folium.LayerControl())
    return mapViz

Import the Landsat 8 data for the area between São Salvador and Peixe Angical

In [4]:
## Import the Landsat 8 data for the area between São Salvador and Peixe Angical
## Import only the data with < 40% cloud cover

l8SasaPean = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')\
          .filter(ee.Filter.eq('WRS_PATH', 222))\
          .filter(ee.Filter.eq('WRS_ROW', 69))\
          .filterDate('2017-07-01', '2018-06-30')\
          .filterMetadata('CLOUD_COVER', 'less_than', 40)

count = l8SasaPean.size()
print('Image count:', count.getInfo())
print(l8SasaPean.getInfo())

Image count: 13
{'type': 'ImageCollection', 'bands': [], 'id': 'LANDSAT/LC08/C01/T1_SR', 'version': 1615115802565245, 'properties': {'system:visualization_0_min': '0.0', 'type_name': 'ImageCollection', 'visualization_1_bands': 'B5,B4,B3', 'thumb': 'https://mw1.google.com/ges/dd/images/LANDSAT_SR_thumb.png', 'visualization_1_max': '30000.0', 'description': '<p>This dataset is the atmospherically corrected\nsurface reflectance from  the Landsat 8 OLI/TIRS sensors.\nThese images contain 5 visible and near-infrared (VNIR) bands and\n2 short-wave infrared (SWIR) bands processed to orthorectified surface\nreflectance, and two thermal infrared (TIR) bands processed to orthorectified\nbrightness temperature</p><p>These data have been atmospherically corrected using\n<a href="https://prd-wret.s3.us-west-2.amazonaws.com/assets/palladium/production/atoms/files/LSDS-1368_L8_C1-LandSurfaceReflectanceCode-LASRC_ProductGuide-v3.pdf">LaSRC</a>\nand includes a cloud, shadow, water and snow mask produce

Get unique dates for each image in the collection

In [5]:
def ymdList(imgcol):
    def iter_func(image, newlist):
        date = ee.Number.parse(image.date().format("YYYYMMdd"));
        newlist = ee.List(newlist);
        return ee.List(newlist.add(date).sort())
    ymd = imgcol.iterate(iter_func, ee.List([]))
    return list(ee.List(ymd).reduce(ee.Reducer.frequencyHistogram()).getInfo().keys())

ymd = ymdList(l8SasaPean)

from pandas import DataFrame

ymdDf = DataFrame(ymd, columns=['date'])

print(ymdDf)

        date
0   20170715
1   20170731
2   20170816
3   20170901
4   20170917
5   20171019
6   20171120
7   20180123
8   20180328
9   20180429
10  20180515
11  20180531
12  20180616


Get sun angle information from metadata

In [6]:
## Get sun angle info
sunAz = l8SasaPean.aggregate_array('SOLAR_AZIMUTH_ANGLE')
print(sunAz.getInfo())

sunZen = l8SasaPean.aggregate_array('SOLAR_ZENITH_ANGLE')
print(sunZen.getInfo())

## Change list into dataframe
from pandas import DataFrame
import pandas as pd

sunAzDf = DataFrame(sunAz.getInfo(), columns=['solar_azimuth'])
sunZenDf = DataFrame(sunZen.getInfo(), columns=['solar_zenith'])

frames = [sunAzDf, sunZenDf]

sunAngles = pd.concat(frames, axis=1)
print(sunAngles)

[41.201656, 44.603764, 49.145565, 55.083778, 62.930813, 86.018692, 109.071152, 106.020981, 63.799332, 46.45969, 41.515007, 38.957207, 38.370541]
[45.534126, 43.23679, 39.761745, 35.508755, 31.049667, 24.623917, 25.051178, 31.107407, 34.019474, 39.332912, 42.298904, 44.779724, 46.276974]
    solar_azimuth  solar_zenith
0       41.201656     45.534126
1       44.603764     43.236790
2       49.145565     39.761745
3       55.083778     35.508755
4       62.930813     31.049667
5       86.018692     24.623917
6      109.071152     25.051178
7      106.020981     31.107407
8       63.799332     34.019474
9       46.459690     39.332912
10      41.515007     42.298904
11      38.957207     44.779724
12      38.370541     46.276974


Add water mask data

In [19]:
## Import gdal to read in raster
from osgeo import gdal

h2oMask = gdal.Open("./drive/MyDrive/Colab Notebooks/sasa_pean_wtr_msk_2018.tif")